In [ ]:
import os
os.chdir("..")

In [ ]:
from src.data.data_process import DataReg
import polars as pl
import arviz as az
import matplotlib.dates as mdates
import numpy as np
import geopandas as gpd
import causalpy as cp
import pandas as pd
from shapely import wkt
import matplotlib.pyplot as plt
dr = DataReg()

In [ ]:
dr.conn.sql("SELECT")

In [ ]:
pr_zips = gpd.GeoDataFrame(dr.make_spatial_table().df())
pr_zips["geometry"] = pr_zips["geometry"].apply(wkt.loads)
pr_zips = pr_zips.set_geometry("geometry")
pr_zips["zipcode"] = pr_zips["zipcode"].astype(str)
pr_zips

In [ ]:
result_spatial = cp.DifferenceInDifferences(
    df_long[df_long.year.isin([-0.5, 0.5])],
    formula="bib ~ 1 + district * post_treatment + spatial_lag",  # Include spatial lag
    time_variable_name="post_treatment",
    group_variable_name="district",
    model=cp.pymc_models.LinearRegression(
        sample_kwargs={"target_accept": 0.98, "random_seed": seed}
    ),
)
